# Name & Z-ID

Juviny Noriega

Z1917876

ASSIGNMENT 6



# 1. Product Classes (20 pts)


In [185]:
# ''Create four classes related to the products your online store 
# will sell: StoreItem, HouseholdItem, GroceryItem, BulkItem''


class StoreItem:
    def __init__(self,name: chr,sku: int,price: float):
        self.name=name
        self.sku=sku
        self.price=price
        #base class
        
    def __str__(self):
        return f"{self.name} ({self.sku}) @ {self.price}" # dunder method    
        
class GroceryItem(StoreItem):   
    def __init__(self,name,sku,price,nutrition={}):
        super().__init__(name,sku,price)
        
        self.nutrition=nutrition
        
    def __str__(self):
        return super().__str__()
        
class HouseholdItem(StoreItem):
    def __init__(self,name,sku,price,brand_name: chr):
        super().__init__(name,sku,price)
        
        self.brand_name=brand_name
    def __str__(self):
        return super().__str__()
        
class BulkItem(GroceryItem):
    def __init__(self,name,sku,price,units: chr):
        super().__init__(name,sku,price)
        
        self.units=units
    def __str__(self):
        return f"{self.name} ({self.sku}) @ {self.price}/{self.units}" # dunder method
        
        

        

        


        
    

# 2. Inventory Class (20 pts)

Create an Inventory class that stores both the items the store carries and the amount of each item that the store has in stock

In [186]:
class Inventory():
    def __init__(self):
        self.items = {}  # Stores items with SKU as key and (item object, quantity) as value

    def add_items(self, item, amount):
        if item.sku in self.items:
            self.items[item.sku] = (item, self.items[item.sku][1] + amount)
        else:
            self.items[item.sku] = (item, amount)

    def remove_items(self, item, amount):
        if item.sku not in self.items or self.items[item.sku][1] < amount:
            raise Exception(f"Cannot remove {amount} of {item.name}; not enough in inventory or not found.")
        self.items[item.sku] = (item, self.items[item.sku][1] - amount)

    def find_item(self, sku):
        if sku not in self.items:
            raise KeyError(f"No item found with SKU {sku}.")
        return self.items[sku][0]

    def has_enough(self, item, amount):
        return item.sku in self.items and self.items[item.sku][1] >= amount

    def __str__(self):
        inventory_summary = ""
        for sku, (item, quantity) in self.items.items():
            inventory_summary += f"{quantity} {item}\n"
        return inventory_summary.strip()
                
                
        
    
    
                
            
            
        

        

#  3. ShoppingCart Class (20 pts)

In [187]:
class ShoppingCart(Inventory):
    def __init__(self, products={}):
        self.products=products
    'It should have add_items and remove_items methods that take a StoreItem object and an amount and adds or removes'
    def add_items(self, item, amount=1):
        # Correct handling for storing items as (item, amount)
        self.products[item.sku] = (item, self.products.get(item.sku, (None, 0))[1] + amount)
    def remove_items(self, item: StoreItem|BulkItem|HouseholdItem|GroceryItem, amount=1):
        if item.sku in self.products:
            self.products[item] -= amount # reducing by amount
            if self.products[item] < 0: #ensures you cannot have less than 0 in quantity
                self.products[item]==0
                raise Exception("Cannot remove more items than in inventory , Quantity has been reset to 0")
        else:
            raise Exception('Cannot remove an item that does not exist')
    
    
    'dd a method that returns a human-readable string with all the items in the cart and their amounts'
    def __str__(self):
        result = ""
        for sku, (item, quantity) in self.products.items():
            # Use the __str__ method of the item to get a human-readable description
            item_description = str(item)  # This calls the __str__ method of StoreItem or its subclasses
            result += f"{quantity} {item_description}\n"
        return result
    
    'add support for += and -= operators that will add or remove a single (1) item (or unit) from the cart'
    def __iadd__(self, item: StoreItem|BulkItem|HouseholdItem|GroceryItem):
        self.add_items(item, 1)
        return self

    def __isub__(self, item: StoreItem|BulkItem|HouseholdItem|GroceryItem):
        self.remove_items(item, 1)
        return self
    'add a cost method that computes the total cost for all of the items in the cart'
    def cost(self):
        total_cost = 0
        for item, amount in self.inventory.items():
            total_cost += item.price * amount
        return total_cost

# 4. Store Class (20 pts)

The Store class will keep track of both the inventory and the customers shopping.

In [188]:
class Store:
    max_carts = 10
    carts_in_use = 0

    def __init__(self):
        self.inventory = Inventory()
        self.active_carts = []

    'class method set_max_carts that sets this variable, raising an exception if this value is not a non-negative integer'
    @classmethod
    def set_max_carts(cls, max_carts):
        if not isinstance(max_carts, int) or max_carts < 0:
            raise ValueError("Max carts must be a non-negative integer.")
        cls.max_carts = max_carts
    'get_cart should then raise an exception if the store has too many carts are in use.' 
    def get_cart(self):
        if len(self.active_carts) >= Store.max_carts:
            raise Exception("Maximum number of carts in use.")
        cart = ShoppingCart()
        self.active_carts.append(cart)
        return cart

    def checkout(self, cart):
        'checkout method that checks the inventory to make sure the number of items being bought in the shopping cart are available and then deducts the quantities selected in the shopping cart from the inventory'
        total_cost = 0
        for sku, (item, amount) in cart.products.items():  # Correctly access the (item, amount) tuple
            if not self.inventory.has_enough(item, amount):
                raise Exception(f"Not enough stock for item {item.name}.")
            total_cost += item.price * amount
            self.inventory.remove_items(item, amount)
        self.active_carts.remove(cart)
        return total_cost
        
        

# 6. Summary and Testing
The final list of classes and methods to be added. Note that all classes should have constructors that properly initialize objects, and the instance fields are not listed here.

In [190]:
store = Store()
store.inventory.add_items(GroceryItem("Cheerios", 38942, 3.99, {'calories': 200, 'fat': 0}), 10)
store.inventory.add_items(HouseholdItem("Trash Bags", 38902, 5.99, "Hefty"), 30)
store.inventory.add_items(BulkItem("Bananas", 4011, 0.59, "lb"), 20)
store.inventory.add_items(BulkItem("Lettuce", 3982, 2.99, "head"), 25)
store.inventory.add_items(GroceryItem("Oreos", 27894, 3.29), 1)

print("Inventory:")
print(store.inventory)

my_cart = store.get_cart()
my_cart.add_items(store.inventory.find_item(4011), 3)
my_cart.add_items(store.inventory.find_item(38902))
my_cart.add_items(store.inventory.find_item(27894), 1)
print("Cart:")
print(my_cart)

total_cost = store.checkout(my_cart)
print(f"You spent ${total_cost}")

print("Updated Inventory:")
print(store.inventory)


my_cart = store.get_cart()
my_cart.add_items(store.inventory.find_item(4013), 3) # raises your  exception
my_cart.add_items(store.inventory.find_item(38902), 1.5) # raises your exception
store.inventory.remove_items(store.inventory.find_item(27894), 3) # raises your exceptio

Inventory:
10 Cheerios (38942) @ 3.99
30 Trash Bags (38902) @ 5.99
20 Bananas (4011) @ 0.59/lb
25 Lettuce (3982) @ 2.99/head
1 Oreos (27894) @ 3.29
Cart:
6 Bananas (4011) @ 0.59/lb
2 Trash Bags (38902) @ 5.99
2 Oreos (27894) @ 3.29



Exception: Not enough stock for item Oreos.